# Kết nối Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Đọc dữ liệu

In [ ]:
import os
import pandas as pd

In [ ]:
%cd /content/drive/MyDrive/Data Mining Field-oriented/data/consolidated/topic

In [ ]:
file_name = f"Pháp_luật.tsv"
try:
    df = pd.read_csv(file_name, sep="\t", encoding="utf-8")
except FileNotFoundError:
    print(f"File {file_name} not found.")

# Xây dựng chủ đề ẩn bằng LDA

In [ ]:
from gensim import corpora
from gensim.models import LdaModel

!pip3 install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

## Loại bỏ nhiễu khỏi văn bản

In [ ]:
import re

def preprocess(text):
    text = str(text)
    pattern_keep = r'(?<=[a-zA-Z0-9])(\.|,)(?=[a-zA-Z0-9])|(?<=[a-zA-Z])\.(?=\s|,|$)'

    text = re.sub(pattern_keep, r'\1', text)

    pattern_remove = r'[.,](?=\s|$|[^a-zA-Z0-9])'
    isbn_pattern = r'\b(?:\d{9}[\dXx]|\d{13})\b'
    ip_pattern = r'\b(?:\d{1,3}\.){3}\d{1,3}\b|\b(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}\b'

    cleaned_text = re.sub(pattern_remove, ' ', text)
    cleaned_text = re.sub(isbn_pattern, ' ', cleaned_text)
    cleaned_text = re.sub(ip_pattern, ' ', cleaned_text)

    cleaned_text = re.sub(r'\s{2,}', ' ', cleaned_text)
    cleaned_text = re.sub(r'[/-]', ' ', cleaned_text)
    cleaned_text = re.sub(r'\b\d+\b', '', cleaned_text)
    cleaned_text = re.sub(r'\s{2,}', ' ', cleaned_text)

    return cleaned_text.strip().split()

## Tạo từ điển và ma trận Bag-of-Words

In [ ]:
df['content'] = df['content'].astype(str)
documents = df['content'].apply(lambda x: x.split()).tolist()

dictionary = corpora.Dictionary(documents)
print(f"Số lượng từ trong từ điển: {len(dictionary)}")

corpus = [dictionary.doc2bow(doc) for doc in documents]
print(f"Ví dụ Bag-of-Words cho tài liệu đầu tiên: {corpus[0]}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
 and should_run_async(code)
Số lượng từ trong từ điển: 46612

Ví dụ Bag-of-Words cho tài liệu đầu tiên: [(0, 1), (1, 16), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 3), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 2), (20, 1), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 2), (27, 2), (28, 2), (29, 2), (30, 1), (31, 3), (32, 2), (33, 1), (34, 1), (35, 2), (36, 1), (37, 1), (38, 1), (39, 1), (40, 2), (41, 1), (42, 1), (43, 1), (44, 2), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 12), (52, 1), (53, 1), (54, 1), (55, 2), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 3), (

## Mô hình LDA

In [ ]:
num_topics = 3

lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

print("\nChủ đề LDA:")
for idx, topic in lda_model.print_topics(-1):
    print(f"Chủ đề {idx}:\n{topic}\n")


Chủ đề LDA:
Chủ đề 0:
0.014*"không" + 0.012*"công_ty" + 0.011*"tiền" + 0.011*"thông_tin" + 0.010*"số" + 0.010*"người" + 0.010*"bị_cáo" + 0.009*"đồng" + 0.009*"năm" + 0.007*"vụ"

Chủ đề 1:
0.029*"công_an" + 0.018*"đối_tượng" + 0.014*"điều_tra" + 0.013*"người" + 0.012*"tỉnh" + 0.012*"ma_tuý" + 0.011*"huyện" + 0.009*"cơ_quan" + 0.008*"tiền" + 0.007*"cảnh_sát"

Chủ đề 2:
0.034*"iphone" + 0.020*"người" + 0.014*"không" + 0.014*"apple" + 0.009*"pro" + 0.008*"tính_năng" + 0.007*"giá" + 0.007*"điện_thoại" + 0.007*"camera" + 0.006*"năm"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



## Đánh giá

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

def calculate_perplexity(lda_model, corpus):
    return lda_model.log_perplexity(corpus)

def calculate_coherence(lda_model, texts, dictionary, coherence_type='c_v'):
    coherence_model = CoherenceModel(
        model=lda_model,
        texts=texts,
        dictionary=dictionary,
        coherence=coherence_type
    )
    return coherence_model.get_coherence()

In [ ]:
results = []

num_topic = 3
lda_model = LdaModel(corpus, num_topics=num_topic, id2word=dictionary, passes=5)

# perplexity_score = calculate_perplexity(lda_model, corpus)

# coherence_score = calculate_coherence(lda_model, documents, dictionary, coherence_type='c_v')

results.append({
    "num_topics": num_topic,
    # "perplexity": perplexity_score,
    # "coherence_score": coherence_score
})

## Thống kê chủ đề

### Thống kê theo số lượng

In [ ]:
from collections import Counter

def get_stat(lda_model, corpus):
    stat = []
    for doc_bow in corpus:
        topic_probs = lda_model.get_document_topics(doc_bow)
        stat_count = max(topic_probs, key=lambda x: x[1])[0]
        stat.append(stat_count)
    return stat

stat = get_stat(lda_model, corpus)

topic_counts = Counter(stat)

print("\nSố lượng tài liệu thuộc mỗi chủ đề:")
for topic, count in topic_counts.items():
    print(f"Chủ đề {topic}: {count} tài liệu")

print(f"\nTổng số tài liệu: {sum(topic_counts.values())}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

Số lượng tài liệu thuộc mỗi chủ đề:
Chủ đề 2: 2909 tài liệu
Chủ đề 0: 3578 tài liệu
Chủ đề 1: 4987 tài liệu

Tổng số tài liệu: 11474



### Lấy ra 200 từ nổi bật

In [ ]:
%cd /content/drive/MyDrive/Data Mining Field-oriented/stats/topic/subtopic

/content/drive/.shortcut-targets-by-id/1RRu-R6hm1MkKpo2-XcloxTBP7mzJuoJU/KPDLHLV /stats


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def export_topics_to_csv(lda_model, num_topics, dictionary, output_file):
    topics = {}
    for topic_id in range(num_topics):
        topic_words = lda_model.show_topic(topic_id, topn=200)
        topics[f"Chủ đề {topic_id}"] = [word for word, weight in topic_words]

    topic_df = pd.DataFrame.from_dict(topics, orient="index").transpose()

    topic_df.to_csv(output_file, index=False, encoding="utf-8-sig")

output_file = "topic_words_Pháp_luật.csv"
export_topics_to_csv(lda_model, num_topics, dictionary, output_file)

### Thống kê tài liệu theo các chủ đề ẩn

In [ ]:
def get_documents_topic_distribution_with_content(lda_model, corpus, num_topics, content_list, top_n=5):
    data = []
    for doc_index, doc_bow in enumerate(corpus):
        topic_probs = lda_model.get_document_topics(doc_bow, minimum_probability=0)
        topic_probs_sorted = sorted(topic_probs, key=lambda x: x[1], reverse=True)[:top_n]

        row = {"Tài liệu": f"Tài liệu {doc_index + 1}", "Nội dung": content_list[doc_index]}
        for i, (topic_id, prob) in enumerate(topic_probs_sorted):
            row[f"Chủ đề {i + 1}"] = f"Chủ đề {topic_id} ({prob:.2%})"
        data.append(row)
    return pd.DataFrame(data)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
content_list = df['content'].tolist()

top_n_topics = 5
documents_topic_df_with_content = get_documents_topic_distribution_with_content(
    lda_model, corpus, num_topics, content_list, top_n=top_n_topics
)

documents_with_content_output_file = "stats_for_Pháp_luật.csv"
documents_topic_df_with_content.to_csv(documents_with_content_output_file, index=False, encoding="utf-8-sig")